In [3]:
pip install pandas fastparquet


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import glob
import pandas as pd

In [5]:
base_directory_path = '/Users/macbookpro/Downloads/Edge IIoTset/Edge-IIoTset dataset/'
attack_directory = 'Attack traffic'
normal_directory = 'Normal traffic'

# csv_files = [file for file in os.listdir(base_directory_path+attack_directory) if file.endswith('.csv')]
csv_files = []
csv_files.extend(glob.glob(base_directory_path+attack_directory+'/*.csv'))
csv_files.extend(glob.glob(base_directory_path+normal_directory+'/**/*.csv'))

dfs = []

for file in csv_files:
    df = pd.read_csv(file, low_memory=False)
    dfs.append(df)

# Concatenate all Dataframes together
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.head()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,2021 23:08:32.886770000,0,0,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Uploading
1,2021 23:08:50.404118000,192.168.0.170,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Uploading
2,2021 23:08:50.405050000,192.168.0.128,192.168.0.170,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Uploading
3,2021 23:08:50.405398000,192.168.0.128,192.168.0.170,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Uploading
4,2021 23:08:50.409978000,192.168.0.170,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Uploading


In [16]:
print("Shape before deduplication: ", merged_df.shape)

# Remove duplicates
merged_df.drop_duplicates(inplace=True)

print("Shape after deduplication: ", merged_df.shape)

Shape before deduplication:  (20939622, 63)
Shape after deduplication:  (19230495, 63)


In [9]:
# Save merged data to file
merged_df.to_csv('merged_edge_iiotset.csv', index=False)

In [10]:
print(f"Successfully merged {len(csv_files)} files into 'merged_edge_iiotset.csv'")

Successfully merged 24 files into 'merged_edge_iiotset.csv'


In [11]:
merged_df["frame.time"] = pd.to_datetime(df["frame.time"], errors="coerce")

/var/folders/zg/s7wmysys2h33l8v23xsdpdwc0000gn/T/ipykernel_2876/3573307581.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df["frame.time"] = pd.to_datetime(df["frame.time"], errors="coerce")


In [14]:
merged_df["ip.dst_host"] = merged_df["ip.dst_host"].astype(str)

In [28]:
object_columns = merged_df.select_dtypes(include="object").columns

In [29]:
for col in object_columns:
    merged_df[col] = merged_df[col].astype(str)

In [30]:
# Save to Parquet
merged_df.to_parquet("edge_iiotset.parquet", index=False)